In [1]:
# If you run on colab uncomment the following line
#!pip install git+https://github.com/clementchadebec/benchmark_VAE.git

In [2]:
import torch
import torchvision.datasets as datasets

%load_ext autoreload
%autoreload 2

In [13]:
mnist_trainset = datasets.MNIST(root='../../data', train=True, download=True, transform=None)

train_dataset = mnist_trainset.data[:-50000].reshape(-1, 1, 28, 28) / 255.
eval_dataset = mnist_trainset.data[-10000:].reshape(-1, 1, 28, 28) / 255.

In [14]:
from pythae.models import VQVAE, VQVAEConfig
from pythae.trainers import BaseTrainingConfig
from pythae.pipelines.training import TrainingPipeline
from pythae.models.nn.benchmarks.mnist import Encoder_AE_MNIST, Decoder_AE_MNIST

In [32]:
config = BaseTrainingConfig(
    output_dir='my_model',
    learning_rate=1e-4,
    batch_size=100,
    num_epochs=1,
)


model_config = VQVAEConfig(
    latent_dim=16,
    input_dim=(1, 28, 28),
    num_embeddings=10
)

model = VQVAE(
    model_config=model_config,
    encoder=Encoder_AE_MNIST(model_config), 
    decoder=Decoder_AE_MNIST(model_config) 
)

torch.Size([2, 16])
1


In [33]:
pipeline = TrainingPipeline(
    training_config=config,
    model=model
)

In [34]:
pipeline(
    train_data=train_dataset,
    eval_data=eval_dataset
)

Preprocessing train data...
Preprocessing eval data...

Using Base Trainer

Model passed sanity check !

Created my_model/VQVAE_training_2022-01-12_14-24-47. 
Training config, checkpoints and final model will be saved here.

Successfully launched training !



torch.Size([2, 4, 4, 1])
torch.Size([32, 10])
torch.Size([32, 1])
torch.Size([32, 1, 1])
torch.Size([2, 4, 4, 1])
torch.Size([2, 4, 4, 1])
torch.Size([2, 1, 4, 4])
torch.Size([]) torch.Size([2])


Training of epoch 1/1:   0%|          | 0/100 [00:00<?, ?batch/s]

torch.Size([100, 4, 4, 1])
torch.Size([1600, 10])
torch.Size([1600, 1])
torch.Size([1600, 1, 1])
torch.Size([100, 4, 4, 1])
torch.Size([100, 4, 4, 1])
torch.Size([100, 1, 4, 4])
torch.Size([]) torch.Size([100])
torch.Size([100, 4, 4, 1])
torch.Size([1600, 10])
torch.Size([1600, 1])
torch.Size([1600, 1, 1])
torch.Size([100, 4, 4, 1])
torch.Size([100, 4, 4, 1])
torch.Size([100, 1, 4, 4])
torch.Size([]) torch.Size([100])


Training of epoch 1/1:   2%|▏         | 2/100 [00:00<00:09, 10.65batch/s]

...

torch.Size([100, 4, 4, 1])
torch.Size([1600, 10])
torch.Size([1600, 1])
torch.Size([1600, 1, 1])
torch.Size([100, 4, 4, 1])
torch.Size([100, 4, 4, 1])
torch.Size([100, 1, 4, 4])
torch.Size([]) torch.Size([100])
torch.Size([100, 4, 4, 1])
torch.Size([1600, 10])
torch.Size([1600, 1])
torch.Size([1600, 1, 1])
torch.Size([100, 4, 4, 1])
torch.Size([100, 4, 4, 1])
torch.Size([100, 1, 4, 4])
torch.Size([]) torch.Size([100])


Training of epoch 1/1:  32%|███▏      | 32/100 [00:02<00:05, 11.85batch/s]

torch.Size([100, 4, 4, 1])
torch.Size([1600, 10])
torch.Size([1600, 1])
torch.Size([1600, 1, 1])
torch.Size([100, 4, 4, 1])
torch.Size([100, 4, 4, 1])
torch.Size([100, 1, 4, 4])
torch.Size([]) torch.Size([100])


Training of epoch 1/1:  32%|███▏      | 32/100 [00:02<00:05, 11.36batch/s]


KeyboardInterrupt: 

In [ ]:
import os

In [ ]:
last_training = sorted(os.listdir('my_model'))[-1]
trained_model = AE.load_from_folder(os.path.join('my_model', last_training, 'final_model'))

In [ ]:
from pythae.samplers import NormalSampler

In [ ]:
# create normal sampler
normal_samper = NormalSampler(
    model=trained_model
)

In [ ]:
# sample
gen_data = normal_samper.sample(
    num_samples=25
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# show results with normal sampler
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(10, 10))

for i in range(5):
    for j in range(5):
        axes[i][j].imshow(gen_data[i*5 +j].cpu().squeeze(0), cmap='gray')
        axes[i][j].axis('off')
plt.tight_layout(pad=0.)

In [ ]:
from pythae.samplers import GaussianMixtureSampler, GaussianMixtureSamplerConfig

In [ ]:
# set up GMM sampler config
gmm_sampler_config = GaussianMixtureSamplerConfig(
    n_components=10
)

# create gmm sampler
gmm_sampler = GaussianMixtureSampler(
    sampler_config=gmm_sampler_config,
    model=trained_model
)

# fit the sampler
gmm_sampler.fit(train_dataset)

In [ ]:
# sample
gen_data = gmm_sampler.sample(
    num_samples=25
)

In [ ]:
# show results with gmm sampler
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(10, 10))

for i in range(5):
    for j in range(5):
        axes[i][j].imshow(gen_data[i*5 +j].cpu().squeeze(0), cmap='gray')
        axes[i][j].axis('off')
plt.tight_layout(pad=0.)

## ... the other samplers work the same